# Парсер
На вход парсеру подается ссылка на категорию из каталога с изделиями, которые планируется спарсить
https://www.585zolotoy.ru/catalog/

Спарсенные данные сохраняются в csv файл папки data.

In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import csv
from datetime import datetime
import os
from urllib.parse import urljoin

ua = UserAgent()


def request_with_retries(url, params=None, timeout=10, retries=3):
    """
    Возвращает response при успехе или None при неудаче.
    """
    headers = {
        "User-Agent": ua.random,
        "accept": "*/*",
        "accept-language": "ru,en;q=0.9",
        "x-qa-client-type": "WEB",
    }

    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, headers=headers, params=params, timeout=timeout)
            if response.ok:
                return response
            print(f"[{attempt}/{retries}] Ошибка статуса {response.status_code} для {url}")
        except requests.RequestException as e:
            print(f"[{attempt}/{retries}] Ошибка запроса к {url}: {e}")
        time.sleep(5)  # пауза перед следующей попыткой

    print(f"Не удалось получить ответ от {url} после {retries} попыток")
    return None


def get_html(url, retries=3):
    """
    Получаем HTML страницы с повторными попытками.
    Возвращаем текст страницы или None при неудаче.
    """
    response = request_with_retries(url, retries=retries)
    return response.text if response else None

def get_breadcrumbs(soup):
    """
    Получаем 'хлебные крошки' для извлечения категории и подкатегории
    """
    breadcrumbs = []
    breadcrumbs_container = soup.find('ul', class_='breadcrumbs')

    if breadcrumbs_container:
        for item in breadcrumbs_container.find_all('li', class_='flex items-center'):
            link = item.find('a', class_='text-caption-1-medium text-text-secondary')
            if link and link.text:
                breadcrumbs.append(link.text.strip())

    # Определяем категорию и подкатегорию на основе структуры хлебных крошек
    category = breadcrumbs[1] if len(breadcrumbs) > 1 else None
    subcategory = breadcrumbs[2] if len(breadcrumbs) > 2 else None

    return category, subcategory

def parse_product_page(article, retries=3):
    """
    Парсим страницу конкретного товара по артикулу.
    Возвращаем словарь с полями или None, если не удалось загрузить страницу.
    """
    base_url = "https://www.585zolotoy.ru/catalog/products/"
    product_url = urljoin(base_url, article)

    html = get_html(product_url, retries=retries)
    if not html:
        # Не удалось загрузить страницу товара
        return None

    soup = BeautifulSoup(html, "html.parser")

    # Название категорий и подкатегорий получаем из "хлебных крошек"
    category, subcategory = get_breadcrumbs(soup)

    # Название товара
    name_tag = soup.find("h1", class_="ui-h1 text-title-2-medium")
    name = name_tag.get_text(strip=True) if name_tag else None

    # Текущая цена
    price_tag = soup.find("strong", class_="product-price__simple-current")
    price = price_tag.get_text(strip=True) if price_tag else None

    # Старая цена (если есть)
    old_price_tag = soup.find("span", class_="product-price__simple-old")
    old_price = old_price_tag.get_text(strip=True) if old_price_tag else None

    # Скидка (если есть)
    discount_tag = soup.find("span", class_="product-price__simple-discount")
    discount = discount_tag.get_text(strip=True) if discount_tag else None

    # Рейтинг — считаем активные звезды
    rating_tag = soup.select_one("div.active-stars")
    rating = len(rating_tag.find_all("div", class_="icon-svg-box active")) if rating_tag else None

    # Отзывы — пытаемся взять первое слово и проверить
    reviews_tag = soup.find("div", class_="text-caption-1-medium text-text-brand")
    reviews = None
    if reviews_tag:
        text = reviews_tag.get_text(strip=True)
        reviews = text.split()[0] if text else ""

    return {
        "category": category,
        "subcategory": subcategory,
        "name": name,
        "sku": article,
        "price": price,
        "old_price": old_price,
        "discount": discount,
        "rating": rating,
        "reviews": reviews,
        "parsed_date": datetime.now().strftime("%Y-%m-%d"),
        "product_url": product_url,
    }


def fetch_articles(category, page=1, retries=3):
    """
    Получаем артикулы товаров через API.
    Явный цикл собирает articles (читаемость > comprehension).
    Возвращаем (articles, next_page) или ([], None) при ошибке.
    """
    url = "https://www.585zolotoy.ru/api/v3/products/"

    params = {
        'category': category,
        'append': '1',
        'page': str(page)
    }

    response = request_with_retries(url, params=params, retries=retries)
    if not response:
        # Нет ответа от API — возвращаем пустой результат
        return [], None

    try:
        data = response.json()
    except ValueError:
        # Некорректный JSON
        print("Ошибка: не удалось разобрать JSON от API.")
        return [], None

    # Получаем список результатов; если ключа нет — используем пустой список
    results = data.get("results", [])

    # Цикл для формирования списка артикулов
    articles = []
    for item in results:
        article = item.get("article")
        if article:
            articles.append(article)

    # Параметры следующей страницы (если есть)
    next_page = data.get("pagination", {}).get("next_page_params")
    return articles, next_page


def save_to_csv(products, filename="products.csv"):
    """
    Сохраняем список товаров в CSV (utf-8-sig).
    Если список пуст — ничего не сохраняем.
    """
    if not products:
        print("Нет данных для сохранения")
        return

    # Создание отдельной папки для хранения спарсенных данных
    data_folder = "data"
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    # Путь к папке
    filepath = os.path.join(data_folder, filename)

    # Определяем порядок полей по первому элементу
    fieldnames = list(products[0].keys())
    with open(filepath, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(products)
    print(f"Сохранено {len(products)} товаров в {filepath}")


def scrape_category(category, filename, sleep_time=1, retries=3):
    """
    Собираем все товары категории и сохраняем в CSV.
    Параметр retries пробрасывается в сетевые функции.
    """
    page = 1
    products = []

    while True:
        print(f"Сканирование страницы {page} категории '{category}'...")
        articles, next_page = fetch_articles(category, page=page, retries=retries)
        if not articles:
            print("Артикулы не найдены или произошла ошибка — прекращаем.")
            break

        for article in articles:
            product = parse_product_page(article, retries=retries)
            if product:
                # Печатаем текущий URL парсинга
                print(f"Парсинг товара: {product['product_url']}")
                products.append(product)
            else:
                print(f"Пропущен артикул {article} (ошибка загрузки/парсинга).")
            time.sleep(sleep_time)

        if not next_page:
            # Если API не вернул данные о следующей странице, завершаем.
            break
        page += 1

    save_to_csv(products, filename)


if __name__ == "__main__":

    # категория, где около 10 тыс. позиций  - длительность парсинга более 8 часов
    # catalog_url = "https://www.585zolotoy.ru/catalog/rings/"

    # для примера, категория гда мало позиций - длительность парсинга менее минуты
    catalog_url = "https://www.585zolotoy.ru/catalog/rings_for_children/"

    category = catalog_url.strip("/").split("/")[-1]
    filename = f"{category}_{datetime.now().strftime('%Y%m%d')}.csv"

    # Запуск парсера
    scrape_category(category, filename, sleep_time=1, retries=3)

Сканирование страницы 1 категории 'rings_for_children'...
Парсинг товара: https://www.585zolotoy.ru/catalog/products/1418515
Парсинг товара: https://www.585zolotoy.ru/catalog/products/9001215
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863441
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863614
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863592
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863682
Парсинг товара: https://www.585zolotoy.ru/catalog/products/1138520
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863429
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863761
Парсинг товара: https://www.585zolotoy.ru/catalog/products/7863447
Парсинг товара: https://www.585zolotoy.ru/catalog/products/2245486
Сохранено 11 товаров в data/rings_for_children_20250829.csv


# Загрузка данных в PySpark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("zolotoy585_analysis") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Чтение всех CSV из папки 'data'
df = spark.read \
    .option("header", True) \
    .option("sep", ",") \
    .csv("data/*.csv")

df.show(20, truncate=False)

+--------------------------+-------------------------+---------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|category                  |subcategory              |name                                               |sku    |price   |old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------------------------+-------------------------+---------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|Кольца                    |Женские кольца           |Золотое кольцо с сапфиром выращенным и бриллиантами|1978724|12 790 ₽|25 580 ₽ |−50%    |5     |42     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/1978724|
|Кольца                    |Мужские кольца           |Золотое обручальное кольцо                         |9001102|12

In [4]:
print(f"Количество строк в исходном DataFrame: {df.count()}")

Количество строк в исходном DataFrame: 9945


In [5]:
# мой список парсинных категорий

parsed_category = ["Кольца"]
parsed_category

['Кольца']

# Разведочный анализ данных


## Анализ и восстановление поля category

In [6]:
# Уникальные значения category

df.select("category").distinct().show(truncate=False)

+--------------------------+
|category                  |
+--------------------------+
|Кольца                    |
|Подвески                  |
|Серьги                    |
|Религия                   |
|Тренды ювелирных украшений|
|Свадьба                   |
|NULL                      |
+--------------------------+



In [7]:
# отбор невалидных категорий, т.е тех, которые не в списке или с NULL значением

df.filter((F.col("category").isNull()) | (~F.col("category").isin(parsed_category))).show(truncate=False)

+--------------------------+----------------------+------------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|category                  |subcategory           |name                                                  |sku    |price   |old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------------------------+----------------------+------------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|Тренды ювелирных украшений|Ювелирная база        |Золотое кольцо с премиум цирконием                    |9002261|13 990 ₽|27 980 ₽ |−50%    |5     |21     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/9002261|
|Тренды ювелирных украшений|Ювелирная база        |Золотое кольцо с бриллиантом                          |1847403|9 

In [8]:
# попытка восстановить категория и подкатегорию из ключевых слов в названии:
# category = "Кольца" если name = "%кольцо%" или name = "%печатка%"

df = df.withColumn("category",
    F.when(F.lower(F.col("name")).like("%кольцо%"), "Кольца")
     .when(F.lower(F.col("name")).like("%печатка%"), "Кольца")
     .otherwise(F.col("category"))
)

df.filter((F.col("category").isNull()) | (~F.col("category").isin(parsed_category))).show(20, truncate=False)

+--------+----------------+------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|category|subcategory     |name                                            |sku    |price   |old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------+----------------+------------------------------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|Подвески|Золотые подвески|Золотая подвеска с цитрином и фианитами         |9002563|17 095 ₽|75 980 ₽ |−78%    |5     |1      |2025-08-28 |https://www.585zolotoy.ru/catalog/products/9002563|
|Серьги  |Серьги с камнями|Серебряные серьги с имитацией кварца и фианитами|1961954|2 026 ₽ |11 580 ₽ |−83%    |5     |1      |2025-08-29 |https://www.585zolotoy.ru/catalog/products/1961954|
+--------+----------------+------------------

## Анализ поля subcategory

In [9]:
# отбор невалидных подкатегорий, с NULL значениями

df.filter((F.col("subcategory").isNull())).show(truncate=False)

+--------+-----------+-----------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|category|subcategory|name                         |sku    |price   |old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------+-----------+-----------------------------+-------+--------+---------+--------+------+-------+-----------+--------------------------------------------------+
|Кольца  |NULL       |Золотое обручальное кольцо   |7209694|42 927 ₽|156 090 ₽|−72%    |5     |50     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/7209694|
|Кольца  |NULL       |Золотое обручальное кольцо   |2792739|31 400 ₽|62 790 ₽ |−50%    |5     |40     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/2792739|
|Кольца  |NULL       |Золотое обручальное кольцо   |4025313|29 452 ₽|107 100 ₽|−73%    |5     |58     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/4025313

Однозначно, по ключевому слову в имени, восстановить поддкатегорию не удасться

## Анализ поля sku

In [10]:
# Попытка преобразовать sku в число, если не получается - оставляем строку

df.filter(F.col("sku").cast("int").isNull()).show()

+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
|category|subcategory|name|sku|price|old_price|discount|rating|reviews|parsed_date|product_url|
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+



## Анализ поля price и old_price

In [11]:
# Последний символ каждой строки price и находим уникальные

(
    df.select(
    F.substring(F.col("price"), -1, 1).alias("last_symbol"))
    .distinct().show()
)

+-----------+
|last_symbol|
+-----------+
|          ₽|
|       NULL|
+-----------+



In [13]:
print("Все строки содержащие отрицательные значения цен")

df.filter(
    (F.col('price') < 0) |
    (F.col('old_price') < 0)
).show(truncate=False)

Все строки содержащие отрицательные значения цен
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
|category|subcategory|name|sku|price|old_price|discount|rating|reviews|parsed_date|product_url|
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+



Есть товары без цены (недоступные для заказа), без указания скидки, без указания старой цены

<!-- Есть товары с ценой NULL (недоступные для заказа), без указания старой цены без скидки -->

# 3. Преобразование и очистка данных

In [14]:
# очистка и преобразование цены
# Удаляем все не-цифровые символы
df = df.withColumn("price",
                  F.regexp_replace(F.col("price"), r'[^\d]', '').cast('int')) \
       .withColumn("old_price",
                  F.regexp_replace(F.col("old_price"), r'[^\d]', '').cast('int'))

In [15]:
# очистка и преобразование скидки

def clean_discount(column):
    return F.when(column.isNotNull(),
                 F.abs(  # значение по модулю
                     F.replace(
                         F.replace(column, F.lit("−"), F.lit("-")),
                         F.lit("%"), F.lit("")
                     ).cast("int")
                 )
         ).otherwise(F.lit(None))

df = df.withColumn("discount", clean_discount(F.col("discount")))

In [16]:
# очистка и преобразование просмотров

def clean_reviews(column):
    return F.when(column.isNotNull(),
                 F.replace(column, F.lit("Без"), F.lit("0")).cast("int")
         ).otherwise(F.lit(None))

df = df.withColumn("reviews", clean_reviews(F.col("reviews")))

In [17]:
df.show(5, truncate=False)

+--------+--------------+---------------------------------------------------+-------+-----+---------+--------+------+-------+-----------+--------------------------------------------------+
|category|subcategory   |name                                               |sku    |price|old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------+--------------+---------------------------------------------------+-------+-----+---------+--------+------+-------+-----------+--------------------------------------------------+
|Кольца  |Женские кольца|Золотое кольцо с сапфиром выращенным и бриллиантами|1978724|12790|25580    |50      |5     |42     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/1978724|
|Кольца  |Мужские кольца|Золотое обручальное кольцо                         |9001102|12347|44900    |73      |5     |46     |2025-08-28 |https://www.585zolotoy.ru/catalog/products/9001102|
|Кольца  |Широкие кольца|Золотое кольцо с бриллиантами 

In [ ]:
print("Количество null значений после восстановления:")

df.select([F.sum(F.isnull(c).cast("int")).alias(c) for c in df.columns]).show()

Количество null значений после восстановления:
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
|category|subcategory|name|sku|price|old_price|discount|rating|reviews|parsed_date|product_url|
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+
|       0|         35|   0|  0| 2039|     2047|    2047|     0|      0|          0|          0|
+--------+-----------+----+---+-----+---------+--------+------+-------+-----------+-----------+



In [ ]:
print("Пример строк, содержащие NULL значения")

df.filter(
    F.col('subcategory').isNull() |
    F.col('price').isNull() |
    F.col('old_price').isNull() |
    F.col('discount').isNull()
).show(truncate=False)

Все строки содержащие NULL значения
+--------+-------------------------+-----------------------------------------------------+-------+-----+---------+--------+------+-------+-----------+--------------------------------------------------+
|category|subcategory              |name                                                 |sku    |price|old_price|discount|rating|reviews|parsed_date|product_url                                       |
+--------+-------------------------+-----------------------------------------------------+-------+-----+---------+--------+------+-------+-----------+--------------------------------------------------+
|Кольца  |Ювелирная база           |Золотое кольцо с бриллиантом                         |1847403|9990 |NULL     |NULL    |5     |235    |2025-08-28 |https://www.585zolotoy.ru/catalog/products/1847403|
|Кольца  |Ювелирная база           |Золотое кольцо с бриллиантами                        |1730487|16990|NULL     |NULL    |5     |55     |2025-08-28 |https:

Вроде норм, можно грузить в базу

# Настройка подключения к БД

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
pd.set_option('display.max_columns', None)

# Параметры подключения к БД

conn_info = {
    'dbname': 'my_db',
    'user': 'my_user',
    'password': 'my_pass',
    'host': 'my_host',
    'port': 6432,
    'options': '-c client_encoding=utf8', # Принудительная установка кодировки
    'client_encoding': 'utf8',

    # Добавляем параметры для устойчивого подключения
    'connect_timeout': 10,
    'keepalives': 1,
    'keepalives_idle': 30,
    'keepalives_interval': 10,
    'keepalives_count': 3,
}

Моя обёртка для работы с PostgreSQL через psycopg2

In [21]:
class Postgres:
    # Простая обёртка для работы с PostgreSQL через psycopg2
    def __init__(self, config):
        """
        Инициализация подключения к базе.
        :param config: словарь с параметрами подключения
        """
        try:
            self._conn = psycopg2.connect(**config)
            self._conn.set_client_encoding('UTF8')  # кодировка UTF-8
        except Exception as e:
            print("Ошибка подключения:", e)
            raise

    def execute(self, query, params=None):
        """
        Выполнить запрос без возврата данных (INSERT/UPDATE/DELETE).
        :param query: SQL-запрос
        :param params: кортеж или список параметров
        """
        with self._conn.cursor() as cur:
            cur.execute(query, params)
        self._conn.commit()  # зафиксировать изменения

    def fetchall(self, query, params=None):
        """
        Выполнить SELECT и вернуть все строки.
        :param query: SQL-запрос
        :param params: кортеж или список параметров
        :return: список словарей (RealDictCursor)
        """
        with self._conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(query, params)
            return cur.fetchall()

    def close(self):
        """
        Закрыть подключение к базе.
        """
        self._conn.close()

Обертка в функции команд по взаимадействия с БД

In [22]:
# функция выборки данных

def SELECT(sql, config, params=None):
    """
    Выполнить SELECT-запрос и вернуть результат в виде DataFrame.
    :param sql: SQL-запрос
    :param config: словарь с параметрами подключения
    :param params: кортеж или список параметров (по умолчанию None)
    :return: pandas.DataFrame с данными
    """
    conn = Postgres(config)
    try:
        rows = conn.fetchall(sql, params)
        df = pd.DataFrame(rows)
    finally:
        conn.close()
    return df

In [23]:
# функция выполнения команд

def EXECUTE(sql, config, params=None, autocommit=False):
    """
    Выполняет SQL-запросы с поддержкой многострочных команд (например, процедур).

    :param sql: SQL-запрос или набор запросов
    :param config: Конфигурация подключения к БД
    :param params: Параметры для запроса (опционально)
    :param autocommit: Если True, выполняет команды вне транзакции (для DDL-команд)
    """
    conn = Postgres(config)
    conn._conn.autocommit = autocommit

    try:
        if '$$' in sql:
            conn.execute(sql, params)
            print("Успешно выполнена процедура/функция.")
        else:
            statements = [stmt.strip() for stmt in sql.split(';') if stmt.strip()]
            for idx, statement in enumerate(statements, start=1):
                conn.execute(statement, params)
                print(f"Успешно выполнена команда {idx}: {statement[:60]}...")
    except Exception as e:
        print(f"Ошибка выполнения SQL: {e}")
        raise
    finally:
        conn.close()

# Сохранение данных в БД

In [25]:
# Создание таблицы в схеме

create_table = """
DROP TABLE IF EXISTS public.zolotoy;
CREATE TABLE public.zolotoy(
    sku INTEGER NOT NULL PRIMARY KEY,
    category VARCHAR(255),
    subcategory VARCHAR(255),
    name VARCHAR(500) NOT NULL,
    price INTEGER,
    old_price INTEGER,
    discount INTEGER,
    rating INTEGER,
    reviews INTEGER NOT NULL,
    parsed_date DATE NOT NULL,
    product_url TEXT NOT NULL,
    upload_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Ограничения для целостности данных
ALTER TABLE public.zolotoy ADD CONSTRAINT chk_price CHECK (price >= 0 OR price IS NULL);
ALTER TABLE public.zolotoy ADD CONSTRAINT chk_old_price CHECK (old_price >= 0 OR old_price IS NULL);
ALTER TABLE public.zolotoy ADD CONSTRAINT chk_discount CHECK (discount BETWEEN 0 AND 100 OR discount IS NULL);
ALTER TABLE public.zolotoy ADD CONSTRAINT chk_rating CHECK (rating BETWEEN 0 AND 5 OR rating IS NULL);
"""
# выполнение запросов
EXECUTE(create_table, conn_info)

Успешно выполнена команда 1: DROP TABLE IF EXISTS public.zolotoy...
Успешно выполнена команда 2: CREATE TABLE public.zolotoy(
    sku INTEGER NOT NULL PRIMAR...
Успешно выполнена команда 3: -- Ограничения для целостности данных
ALTER TABLE public.zol...
Успешно выполнена команда 4: ALTER TABLE public.zolotoy ADD CONSTRAINT chk_old_price CHEC...
Успешно выполнена команда 5: ALTER TABLE public.zolotoy ADD CONSTRAINT chk_discount CHECK...
Успешно выполнена команда 6: ALTER TABLE public.zolotoy ADD CONSTRAINT chk_rating CHECK (...


In [ ]:
def upload_to_postgres_batch(df_pandas, conn_info, batch_size=100):
    """Пакетная загрузка данных в PostgreSQL"""
    import psycopg2
    from psycopg2.extras import execute_batch
    import pandas as pd

    # SQL запрос для вставки
    insert_sql = """
        INSERT INTO public.zolotoy (
            sku, category, subcategory, name, price, old_price,
            discount, rating, reviews, parsed_date, product_url
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (sku) DO UPDATE SET
            category = EXCLUDED.category,
            subcategory = EXCLUDED.subcategory,
            name = EXCLUDED.name,
            price = EXCLUDED.price,
            old_price = EXCLUDED.old_price,
            discount = EXCLUDED.discount,
            rating = EXCLUDED.rating,
            reviews = EXCLUDED.reviews,
            parsed_date = EXCLUDED.parsed_date,
            product_url = EXCLUDED.product_url
    """

    # Подготавливаем данные
    records = []
    for _, row in df_pandas.iterrows():
        record = (
            row['sku'],
            row['category'],
            row['subcategory'] if pd.notna(row['subcategory']) else None,
            row['name'],
            row['price'] if pd.notna(row['price']) else None,
            row['old_price'] if pd.notna(row['old_price']) else None,
            row['discount'] if pd.notna(row['discount']) else None,
            row['rating'] if pd.notna(row['rating']) else None,
            row['reviews'],
            row['parsed_date'],
            row['product_url']
        )
        records.append(record)

    # Загружаем данные
    try:
        conn = psycopg2.connect(**conn_info)
        cur = conn.cursor()

        # Выполняем пакетную вставку
        execute_batch(cur, insert_sql, records, page_size=batch_size)
        conn.commit()

        print(f"Успешно загружено {len(records)} записей")

    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        if 'conn' in locals() and conn:
            conn.rollback()
        raise

    finally:
        if 'cur' in locals() and cur:
            cur.close()
        if 'conn' in locals() and conn:
            conn.close()

## Запись в БД

In [27]:
# Преобразуем Spark DataFrame в pandas DataFrame

df_pandas = df.toPandas()

In [ ]:
# Загружаем данные

upload_to_postgres_batch(df_pandas, conn_info, batch_size=100)

In [30]:
# Просмотр данных из БД

get_data = """
    SELECT *
    FROM public.zolotoy
    LIMIT 5;
"""
# выполнение запроса
SELECT(get_data, conn_info)

,sku,category,subcategory,name,price,old_price,discount,rating,reviews,parsed_date,product_url,upload_at
0,1593061,Кольца,Кольца из красного золота,Золотое кольцо с бриллиантами,35194,127980,73,5,1.0,2025-08-28,https://www.585zolotoy.ru/catalog/products/159...,2025-08-29 16:09:20.444593
1,1317391,Кольца,Кольца из красного золота,Золотое кольцо с рубином и бриллиантами,51495,205980,75,5,1.0,2025-08-28,https://www.585zolotoy.ru/catalog/products/131...,2025-08-29 16:09:20.444593
2,2004442,Кольца,Мужские кольца,Золотое обручальное кольцо,37548,136540,73,5,34.0,2025-08-28,https://www.585zolotoy.ru/catalog/products/200...,2025-08-29 16:09:20.444593
3,1583329,Кольца,Кольца из красного золота,Золотое кольцо с фианитами,7700,30790,75,5,32.0,2025-08-28,https://www.585zolotoy.ru/catalog/products/158...,2025-08-29 16:09:20.444593
4,1847456,Кольца,Кольца из красного золота,Золотое кольцо с бриллиантами,48495,193980,75,5,7.0,2025-08-28,https://www.585zolotoy.ru/catalog/products/184...,2025-08-29 16:09:20.444593


In [31]:
# Количество записей в БД

get_count_data = """
    SELECT COUNT(*)
    FROM public.zolotoy;
"""

SELECT(get_count_data, conn_info)

,count
0,9945


In [32]:
print(f"Количество строк в исходном DataFrame: {df.count()}")

Количество строк в исходном DataFrame: 9945
